In [2]:
# Assignment 2

In [1]:
# use the Default data to classify default with income+balance+student
library(ISLR)

Warning message:
“package ‘ISLR’ was built under R version 4.0.2”


In [2]:
# split the data in tain and test with one random sample of 80/20
# what do you have to consider here?

print('I need to ensure that the splits contain all relevant categories, for example by checking the count with table().')

table(Default$default, Default$student)

print('For "Default" the prof. said random sampling is OK since "defaulted" is infrequent, but not too infrequent.')


[1] "I need to ensure that the splits contain all relevant categories."
[1] "For \"Default\" the prof. said random sampling is OK since \"defaulted\" is infrequent, but not too infrequent."


     
        No  Yes
  No  6850 2817
  Yes  206  127

In [17]:
# random sample 20% for test set
testindices <- sample(nrow(Default), nrow(Default)/5)

In [18]:
# make new data frame objects for train and test
train.data <- Default[-testindices,]
test.data <- Default[testindices,]

classify this split with
    logistic regression
    linear discriminant analysis
    quadratic discriminant analysis: qda() from MASS
    k-nearest neighbors: knn() from class!

In [19]:
# logistic regression fit
log.fit <- glm(default ~ income+balance+student, family='binomial', train.data)

In [20]:
# LDA fit
library(MASS)
lda.fit <- lda(default ~ income+balance+student, train.data)

In [21]:
# QDA fit
qda.fit <- qda(default ~ income+balance+student, train.data)

In [22]:
# k-nearest fit: different approach: does not build a model, but directly classifies new data
# so this is actually already the prediction
# I use cbind() to provide the predictors, since knn does not provide the "equation" form (y~x)
library(class)
knn.fit <- knn(cbind(as.integer(train.data$student), train.data[,3:4]), cbind(as.integer(test.data$student), test.data[,3:4]), train.data$default, k=1)

In [23]:
summary(log.fit)


Call:
glm(formula = default ~ income + balance + student, family = "binomial", 
    data = train.data)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4042  -0.1510  -0.0593  -0.0221   3.6295  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.037e+01  5.264e-01 -19.709  < 2e-16 ***
income      -4.060e-06  8.988e-06  -0.452  0.65148    
balance      5.609e-03  2.500e-04  22.435  < 2e-16 ***
studentYes  -7.451e-01  2.571e-01  -2.898  0.00375 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 2380.8  on 7999  degrees of freedom
Residual deviance: 1304.3  on 7996  degrees of freedom
AIC: 1312.3

Number of Fisher Scoring iterations: 8


##############

discuss the relevance of predictors
create a confusion matrix
calculate precision, recall and F1-score
discuss differences between the approaches


####### log model

In [24]:
# discuss the relevance of predictors

# summary(log.fit)

print('income is the only slope to be NOT considered different from 0 in this model, so the others seem to be the relevant predictors')

# create a confusion matrix
log.pred <- rep('No', nrow(test.data))
log.pred[predict(log.fit, test.data, type='response')>=.5] = 'Yes'

log.matrix <- table(true=test.data$default, prediction=log.pred)
print('')
print(log.matrix)


# calculate precision, recall and F1-score
precision = log.matrix[2,2]/sum(log.matrix[,2])
recall = log.matrix[2,2]/sum(log.matrix[2,])
F1 = 2*precision*recall/(precision+recall)

print('')
print(paste(precision, recall, F1))



[1] "income is the only slope to be NOT considered different from 0 in this model, so the others seem to be the relevant predictors"
[1] ""
     prediction
true    No  Yes
  No  1935    5
  Yes   39   21
[1] ""
[1] "0.807692307692308 0.35 0.488372093023256"


##########
lda model

In [26]:
# discuss the relevance of predictors

print('normalizing of numerical data can be used to infer predictor strength from model coefficients')
lda.norm.fit <- lda(default ~ scale(income)+scale(balance)+student, train.data)
lda.norm.fit

print('balance is by far the strongest predictor, income looks irrelevant')

# create a confusion matrix
lda.matrix <- table(true=test.data$default, prediction=predict(lda.fit, test.data)$class)
print('')
print(lda.matrix)

# calculate precision, recall and F1-score
precision = lda.matrix[2,2]/sum(lda.matrix[,2])
recall = lda.matrix[2,2]/sum(lda.matrix[2,])
F1 = 2*precision*recall/(precision+recall)

print('')
print(paste(precision, recall, F1))


[1] "normalizing of numerical data can be used to infer predictor strength from model coefficients"


Call:
lda(default ~ scale(income) + scale(balance) + student, data = train.data)

Prior probabilities of groups:
      No      Yes 
0.965875 0.034125 

Group means:
    scale(income) scale(balance) studentYes
No    0.004302731     -0.0656359  0.2900220
Yes  -0.121784621      1.8577605  0.3736264

Coefficients of linear discriminants:
                        LD1
scale(income)   0.009580553
scale(balance)  1.083582645
studentYes     -0.223066817

[1] "balance is by far the strongest predictor, income looks irrelevant"
[1] ""
     prediction
true    No  Yes
  No  1938    2
  Yes   47   13
[1] ""
[1] "0.866666666666667 0.216666666666667 0.346666666666667"


qda model

In [27]:
# discuss the relevance of predictors

print('we have not learned how to deal with this')

# create a confusion matrix
qda.matrix <- table(true=test.data$default, prediction=predict(qda.fit, test.data)$class)
print('')
print(qda.matrix)

# calculate precision, recall and F1-score
precision = qda.matrix[2,2]/sum(qda.matrix[,2])
recall = qda.matrix[2,2]/sum(qda.matrix[2,])
F1 = 2*precision*recall/(precision+recall)

print('')
print(paste(precision, recall, F1))



[1] "we have not learned how to deal with this"
[1] ""
     prediction
true    No  Yes
  No  1936    4
  Yes   43   17
[1] ""
[1] "0.80952380952381 0.283333333333333 0.419753086419753"


In [28]:
# discuss the relevance of predictors

print('in knn, all predictors are equally relevant by default weighted similarly to calculate distances')

# create a confusion matrix
knn.matrix <- table(true=test.data$default, predicted=knn.fit)
print('')
print(knn.matrix)

# calculate precision, recall and F1-score
precision = knn.matrix[2,2]/sum(knn.matrix[,2])
recall = knn.matrix[2,2]/sum(knn.matrix[2,])
F1 = 2*precision*recall/(precision+recall)

print('')
print(paste(precision, recall, F1))






[1] "in knn, all predictors are equally relevant by default weighted similarly to calculate distances"
[1] ""
     predicted
true    No  Yes
  No  1905   35
  Yes   36   24
[1] ""
[1] "0.406779661016949 0.4 0.403361344537815"


In [49]:
## use logistic with 10-fold CV, since it has the highest F1 for test split

In [33]:
library(boot)
cv.fit <- cv.glm(Default, glm(default ~ income+balance+student, family='binomial', Default), K=10)

# prediction error
cv.fit$delta[1]

[1] 0.02140485

In [35]:
# compare to error of single model
mean(test.data$default!=log.pred)

print('CV has minimally lower error than single split. You could also have compared results based on confusion matrix or F1')

[1] 0.022

[1] "CV has minimally lower error than single split. You could also have compared results based on confusion matrix or F1"


In [ ]:
points:

- 10 statement sample/split 
- 05/20 for each model
- 05/20 for each predictor statement
- 05/20 for each confusion matrix and precision/recall/F1
- 10 eval approaches
- 10 for CV
- 10 for interpretation


